<a href="https://colab.research.google.com/github/iamab3/ML-for-Wear-Monitoring-and-Prediction/blob/main/ML_Model_for_Wear_Monitoring_and_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setting up Linear, Ridge and Lasso Regression model for wear monitoring and prediction
# Importing libraries

import numpy as np
import pandas as pd
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
data = pd.read_excel('/content/Sample_Data.xlsx', sheet_name='Sheet3')


In [ ]:
data.head(5)

,S. No.,Inpection Date,New (mm),Worn (mm),Measured (mm),WR (mm)/Day,WR (mm)/kT,WR (mm)/Op.Hr,Predicted EOL Date,Tonnes,Op.Hr
0,1,2020-01-04,295,75,265,0.422535,0.031052,0.021157,2021-03-28 16:00:00.000,6.118697e+06,8980.666667
1,2,2020-07-12,295,75,219,0.291188,0.022768,0.015561,2021-12-14 20:46:07.655,7.174245e+06,10496.955224
2,3,2020-08-19,295,75,177,0.394649,0.030303,0.020504,2021-02-02 19:25:16.168,4.172094e+06,6166.071429
3,4,2020-10-08,295,75,156,0.398281,0.030240,0.020304,2021-04-26 02:24:44.401,3.593807e+06,5352.567568
4,5,2021-03-17,295,75,120,0.343811,0.025403,0.017092,2021-08-10 00:34:55.864,2.325562e+06,3456.425356


In [ ]:
# Convert dates
data['Inpection Date'] = pd.to_datetime(data['Inpection Date'])
data['Predicted EOL Date'] = pd.to_datetime(data['Predicted EOL Date'])

In [ ]:
# Sort by Inspection date
data = data.sort_values(by='Inpection Date').reset_index(drop=True)

In [ ]:
# Create new target columns
data['Days_Since_Last_Inspection'] = data['Inpection Date'].diff().dt.days
data['Days_Since_Last_Inspection'].fillna(0, inplace=True)
data['Next_Inspection_Interval'] = data['Days_Since_Last_Inspection']#.shift(-1)  # target 1
data['Days_to_EOL'] = (data['Predicted EOL Date'] - data['Inpection Date']).dt.days  # target 3

/tmp/ipython-input-941232083.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Days_Since_Last_Inspection'].fillna(0, inplace=True)


In [ ]:
# Drop last row (NaN in target)
data = data.dropna(subset=['Next_Inspection_Interval'])

In [ ]:
# Use these features
features = ['WR (mm)/Day', 'WR (mm)/kT', 'WR (mm)/Op.Hr', 'Tonnes', 'Op.Hr']
X = data[features]

# Targets
y_inspect = data['Next_Inspection_Interval']      # For next inspection date
y_measured = data['Measured (mm)']                # For predicted measurement
y_eol = data['Days_to_EOL']                       # For predicted EOL from inspection date

# Split for evaluation (optional)
X_train, X_test, y_inspect_train, y_inspect_test = train_test_split(X, y_inspect, test_size=0.2, random_state=42)
_, _, y_measured_train, y_measured_test = train_test_split(X, y_measured, test_size=0.2, random_state=42)
_, _, y_eol_train, y_eol_test = train_test_split(X, y_eol, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled_all = scaler.transform(X)  # For final prediction

# ----------------------
# Step 3: Ridge Regression Model
# ----------------------

ridge_inspect = RidgeCV(alphas=[0.1, 1.0, 10.0]).fit(X_train_scaled, y_inspect_train)
ridge_measured = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=2).fit(X_train_scaled, y_measured_train)
ridge_eol = RidgeCV(alphas=[0.1, 1.0, 10.0]).fit(X_train_scaled, y_eol_train)

# Evaluate model
print("Model Performance:\n")

def evaluate_model(name, model, X_test, y_test):
    preds = model.predict(X_test)
    print(f"{name}")
    print(f"  R2: {r2_score(y_test, preds):.2f}")
    print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, preds)):.2f}\n")

evaluate_model("Next Inspection (days)", ridge_inspect, X_test_scaled, y_inspect_test)
evaluate_model("Measured (mm)", ridge_measured, X_test_scaled, y_measured_test)
evaluate_model("Days to EOL", ridge_eol, X_test_scaled, y_eol_test)

# ----------------------
# Step 6: Make Final Prediction (from latest data row)
# ----------------------

from datetime import datetime

# Use last available record
last_row = data.iloc[-1]
last_inspection_date = last_row['Inpection Date']
latest_features = last_row[features].values.reshape(1, -1)
latest_scaled = scaler.transform(latest_features)

# Predict days to next inspection
predicted_gap = ridge_inspect.predict(latest_scaled)[0]
predicted_next_inspection = last_inspection_date + timedelta(days=int(predicted_gap))

# Predict measured (mm)
predicted_measured = ridge_measured.predict(latest_scaled)[0]

# Predict days from next inspection to EOL
predicted_days_to_EOL = ridge_eol.predict(latest_scaled)[0]
predicted_eol_date = predicted_next_inspection + timedelta(days=int(predicted_days_to_EOL))

# ----------------------
# Step 7: Output Predictions
# ----------------------

print("Predicted Future Values:")
print(f"  Last Inspection Date      : {last_inspection_date.date()}")
print(f"  Predicted Next Inspection : {predicted_next_inspection.date()} (+{int(predicted_gap)} days)")
print(f"  Predicted Measured (mm)   : {predicted_measured:.2f} mm")
print(f"  Predicted EOL Date        : {predicted_eol_date.date()} (+{int(predicted_days_to_EOL)} days after inspection)")



Model Performance:

Next Inspection (days)
  R2: nan
  RMSE: 22.63

Measured (mm)
  R2: nan
  RMSE: 35.94

Days to EOL
  R2: nan
  RMSE: 264.87

Predicted Future Values:
  Last Inspection Date      : 2021-03-17
  Predicted Next Inspection : 2021-08-23 (+159 days)
  Predicted Measured (mm)   : 116.77 mm
  Predicted EOL Date        : 2022-01-20 (+150 days after inspection)


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
